### 1. Run imports

In [ ]:
import camelot
import pandas as pd
import numpy as np
import string

### 2. Enter desired file locations

In [ ]:
# copy in current input pdf and desired output csv locations
# make sure / rather than \ , if \  used can change to / or put r before string e.g.r"\computer\file_location\file"

input_pdf = "/home/michael/jupyter_folder/lion_rock_2022_interim_report.pdf"
csv = "/home/michael/jupyter_folder/lion_rock_2022_interim_report_input_7.csv"
csv_output = "/home/michael/jupyter_folder/lion_rock_2022_interim_report_output_7.csv"

# input_pdf = "/home/michael/jupyter_folder/2023 10q3.pdf"
# csv = "/home/michael/jupyter_folder/20230626_input_test_3.csv"
# csv_output = "/home/michael/jupyter_folder/20230626_output_test_3.csv"

### 3. Read in the pdf, assign each table to a dataframe, and export to csv

In [ ]:
# will result in 'IndexError: sequence index out of range' unless max_page_no is changed to correct value, but runs fine...
# ... and exports to csv regardless will only error out after all pages uploaded so feel free to leave at 200
max_page_no = 200

for i in range(1,max_page_no):
    page = str(i)
    tables = camelot.read_pdf(input_pdf, flavor='stream', pages=page, table_areas=['0,780,566,0'], encoding='utf-8')

    no_tables = tables.n
    
    # print below optional just shows camelot is doing what it is supposed to, while running table areas at ['0,780,566,0'] should be one table per page and every page picked up
    print('Page number:', i, ' Number of tables:', no_tables)
    
    for j in range(1, no_tables + 1):
        df = tables[j-1].df
        
#         # below if/else removes a large amount of excess text not in tables from pdf, can be hashed/unhashed and easily changed as required
#         char_max = 30
#         if len(df.columns) > 2:
#             df = df[~(df[0].astype(str).str.len().gt(char_max) | df[1].astype(str).str.len().gt(char_max)| df[2].astype(str).str.len().gt(char_max))]
#         if len(df.columns) > 1:
#             df = df[~(df[0].astype(str).str.len().gt(char_max) | df[1].astype(str).str.len().gt(char_max))]
#         else:
#             df = df[~df[0].astype(str).str.len().gt(char_max)]

        # Create an empty row DataFrame with the same columns as df
        empty_row = pd.DataFrame([[''] * len(df.columns)], columns=df.columns)
        # Concatenate df and empty_rows (as many as desired have put 3 as standard)
        df = pd.concat([df, empty_row, empty_row, empty_row], ignore_index=True)
        

        df.to_csv(csv, mode='a', index=False, header=False)

### results may include error message shown below, this is caused by large blocks of text with words at a distance apart that
### camelot is not sure how to treat them, as long as it keeps running after (it should), this is completely fine
    # C:\Users\winte\anaconda3\lib\site-packages\camelot\parsers\stream.py:365: UserWarning: No tables found in table area 1
    # warnings.warn(f"No tables found in table area {table_idx + 1}")

### 4. Replace em dashes (—) with 0s

In [ ]:
input_csv = open(csv, "r")

# test em dash replace
# input_csv = ''.join([i for i in input_csv]) .replace(b'\xe2\x80\x94'.decode('utf-8'), 'TESTING TESTING')

# original em dash replace 
input_csv = ''.join([i for i in input_csv]) .replace("–", "0")

# added em dash replace
input_csv = ''.join([i for i in input_csv]) .replace("—", "0")

# optional line below to also remove many undesired line breaks in file
input_csv = ''.join([i for i in input_csv]) .replace("$\n", "$")


output_csv = open(csv_output,"w")
output_csv.writelines(input_csv)
output_csv.close()


### 5. (optional) If you are not getting all the information from each page as required, experiment adjusting table areas below and entering new table areas desired above in 3.

In [ ]:
tables = camelot.read_pdf(input_pdf, flavor='stream', pages='1', table_areas=['0,780,566,0'])

camelot.plot(tables[0], kind='contour').show()